### First steps
We create some functions to parse the data and declare some constants that would be useful

In [49]:
# Constants
max_length_name = 20

# function to convert a string to a list of ascii values
def transform_string(string):
   string = string.lower()
   ascii_list = [ord(c) for c in string]
   list_ = ascii_list + [0] * (max_length_name - len(ascii_list))
   return np.array(list_).astype(np.float32)


### Load libraries/API's

In [2]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd


### Build Model

In [29]:
def build_model(vocab_size=255, embedding_dim=256, rnn_units=128):
    #build NN
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=max_length_name))
    model.add(Bidirectional(LSTM(rnn_units, recurrent_dropout=0.2, dropout=0.2)))
    model.add(Dense(1, activation='sigmoid'))

    #Compile
    model.compile(loss='binary_crossentropy', optimizer=Adam(
        learning_rate=0.01), metrics=['accuracy'])

    return model


### Train the model

In [4]:
# load the data
df = pd.read_csv('unificado.csv')
df.head()

,name,sex
0,aaden,1
1,aaliyah,0
2,aamir,1
3,aarav,1
4,aaron,1


In [30]:
model = build_model()

In [27]:

X = np.asarray(df['name'].values.tolist())
Y = np.asarray(df['sex'].values.tolist())

#Tranform names to numbers
X = np.array([transform_string(val) for val in X])

In [34]:
history = model.fit(x=X, y=Y, batch_size=1, epochs=2)
model.save('model_v1.h5')

Epoch 1/2
12312/12312 [==============================] - 1281s 104ms/step - loss: 0.7109 - accuracy: 0.5905
Epoch 2/2
12312/12312 [==============================] - 1274s 103ms/step - loss: 0.6728 - accuracy: 0.6590


In [51]:
def predict(name, model_loaded):
    data = np.array([transform_string(val) for val in name])
    result = model_loaded.predict(data)
    return ['m' if logit > 0.5 else 'f' for logit in result]


In [58]:
test_data = ['Roberto', 'Stephen', 'Allan']
predictions = predict(test_data, model)
print(predictions)


1/1 [==============================] - 0s 234ms/step
['m', 'm', 'm']
